In [1]:
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector

thres = 0.5

def calculate_object_distance(box):
    focal_length = 1000
    known_object_width = 10
    x, y, w, h = box
    distance = (known_object_width * focal_length) / w
    return distance

def calculate_face_distance(pointLeft, pointRight):
    w, _ = detector.findDistance(pointLeft, pointRight)
    W = 6.3
    f = 840
    distance = (W * f) / w
    return distance

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

detector = FaceMeshDetector(maxFaces=10)

classNames = []

config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = 'frozen_inference_graph.pb'

net = cv2.dnn_DetectionModel(weightsPath, config_file)
net.setInputSize(320, 320)
net.setInputScale(1.0 / 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

while True:
    success, img = cap.read()
    if not success:
        break

    classIds, confs, bbox = net.detect(img, confThreshold=thres)
    if len(classIds) != 0:
        for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), bbox):
            distance = calculate_object_distance(box)
            color = (0, 0, 255) if distance < 25 else (0, 255, 0)
            cv2.rectangle(img, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), color, thickness=2)
            cv2.putText(img, f'Obj Dist: {distance:.1f} cm', (box[0], box[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    img, faces = detector.findFaceMesh(img)
    for face in faces:
        pointLeft = face[145]
        pointRight = face[374]
        cv2.line(img, pointLeft, pointRight, (0, 200, 0), 3)
        cv2.circle(img, pointLeft, 5, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, pointRight, 5, (255, 0, 255), cv2.FILLED)
        face_distance = calculate_face_distance(pointLeft, pointRight)
        cvzone.putTextRect(img, f'Face Dist: {int(face_distance)} cm',
                           (face[10][0] - 75, face[10][1] - 50), scale=2)

    cv2.imshow("Combined Output", img)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()
